In [2]:
import time
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
import math
import sys
import os
from Manifold_Analysis import Manifold_analysis
sys.path.append('../../../')
from algos.agents import A2CAgent
from algos.models.actor_critic_cnn import ActorCnn, CriticCnn
from algos.models.actor_critic_cnn_init import ActorCnn_init, CriticCnn_init
from algos.preprocessing.stack_frame import preprocess_frame, stack_frame

# importing required libraries
from mpl_toolkits.mplot3d import Axes3D

from sklearn.datasets import load_iris
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.manifold import MDS
from sklearn.manifold import Isomap
from sklearn.metrics import pairwise_distances

from matplotlib.ticker import NullFormatter
from collections import OrderedDict
from functools import partial
from sklearn import manifold, datasets

import cv2

In [4]:
# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

#env = gym.make('SpaceInvaders-v0',render_mode='human')
env = gym.make('SpaceInvaders-v0')


env.seed(0)

def stack_frames(frames, state, is_new=False):
    frame = preprocess_frame(state, (8, -12, -12, 4), 84)
    frames = stack_frame(frames, frame, is_new)

    return frames
    

INPUT_SHAPE = (4, 84, 84)
ACTION_SIZE = env.action_space.n
SEED =0
GAMMA =0.99           # discount factor
ALPHA=0.0001          # Actor learning rate
BETA =0.0005          # Critic learning rate
UPDATE_EVERY = 100     # how often to update the network 


epsilon_by_epsiode = lambda frame_idx: EPS_END + (EPS_START - EPS_END) * math.exp(-1. * frame_idx /EPS_DECAY)

start_epoch =0
scores = []
scores_window = deque(maxlen=20)
length_trial = 500

Device:  cpu


A.L.E: Arcade Learning Environment (version +978d2ce)
[Powered by Stella]


# LOAD INIT MODEL

# LOAD HOOKS MODEL

In [216]:
def copy_model(model_init,model_hooks,PATH):
    G = []
    for param_tensor in model_hooks.actor_net.state_dict():
        G.append(param_tensor)
    H = []
    for param_tensor in model_hooks.critic_net.state_dict():
        H.append(param_tensor)

    i =0
    for param_tensor in model_init.actor_net.state_dict():
        model_hooks.actor_net.state_dict()[G[i]].data.copy_(model_init.actor_net.state_dict()[param_tensor])
        i+=1
    i =0
    for param_tensor in model_init.critic_net.state_dict():
        model_hooks.critic_net.state_dict()[H[i]].data.copy_(model_init.critic_net.state_dict()[param_tensor])
        i+=1
    
    torch.save({
            'epoch':0,
            'modelA_state_dict': model_hooks.actor_net.state_dict(),
            'modelB_state_dict': model_hooks.critic_net.state_dict(),
            'optimizerA_state_dict': model_hooks.actor_optimizer.state_dict(),
            'optimizerB_state_dict': model_hooks.critic_optimizer.state_dict(),
            'Score':0,
            }, PATH)
    

        

In [217]:
agent_0 = A2CAgent(INPUT_SHAPE, ACTION_SIZE, SEED, device, GAMMA, ALPHA, BETA, UPDATE_EVERY, ActorCnn, CriticCnn)
agent_0_init = A2CAgent(INPUT_SHAPE, ACTION_SIZE, SEED, device, GAMMA, ALPHA, BETA, UPDATE_EVERY, ActorCnn_init, CriticCnn_init)


In [218]:
a = "model_a2c_0_2"
PATH = os.path.join(os.getcwd(),"Models_training_a2c_space_invader",a+ ".pt")
print(PATH)
checkpoint = torch.load(PATH , map_location=torch.device('cpu'))
agent_0_init.actor_net.load_state_dict(checkpoint['modelA_state_dict'])
agent_0_init.critic_net.load_state_dict(checkpoint['modelB_state_dict'])
agent_0_init.actor_optimizer.load_state_dict(checkpoint['optimizerA_state_dict'])
agent_0_init.critic_optimizer.load_state_dict(checkpoint['optimizerB_state_dict'])



agent_0.actor_optimizer.load_state_dict(checkpoint['optimizerA_state_dict'])
agent_0.critic_optimizer.load_state_dict(checkpoint['optimizerB_state_dict'])


/Users/charlottebeylier/Documents/PhD/Atari1.0/Reinforcement-Learning_modif/cgames/02_space_invader/Analysis/Models_training_a2c_space_invader/model_a2c_0_2.pt


In [219]:
PATH = os.path.join(os.getcwd(),"Models_training_a2c_space_invader","agent_a2c_0_2"+ ".pt")
copy_model(agent_0_init,agent_0,PATH)

In [220]:
print(agent_0.actor_net.state_dict()['fc2.0.bias'])
print(agent_0.actor_net.state_dict()['fc2.0.weight'])

tensor([ 0.0092, -0.0320,  0.0407,  0.0175,  0.0115, -0.0174])
tensor([[-0.0305,  0.0434, -0.0298,  ...,  0.0029,  0.0153,  0.0188],
        [-0.0114,  0.0372,  0.0164,  ..., -0.0280,  0.0407,  0.0196],
        [-0.0099, -0.0288, -0.0011,  ...,  0.0354, -0.0200,  0.0275],
        [-0.0239, -0.0246, -0.0034,  ...,  0.0120, -0.0229,  0.0118],
        [-0.0032,  0.0082,  0.0427,  ..., -0.0382,  0.0403, -0.0124],
        [ 0.0288, -0.0233, -0.0295,  ..., -0.0200, -0.0117, -0.0272]])


In [221]:
print(agent_0_init.actor_net.state_dict()['fc.2.bias'])
print(agent_0_init.actor_net.state_dict()['fc.2.weight'])

tensor([ 0.0092, -0.0320,  0.0407,  0.0175,  0.0115, -0.0174])
tensor([[-0.0305,  0.0434, -0.0298,  ...,  0.0029,  0.0153,  0.0188],
        [-0.0114,  0.0372,  0.0164,  ..., -0.0280,  0.0407,  0.0196],
        [-0.0099, -0.0288, -0.0011,  ...,  0.0354, -0.0200,  0.0275],
        [-0.0239, -0.0246, -0.0034,  ...,  0.0120, -0.0229,  0.0118],
        [-0.0032,  0.0082,  0.0427,  ..., -0.0382,  0.0403, -0.0124],
        [ 0.0288, -0.0233, -0.0295,  ..., -0.0200, -0.0117, -0.0272]])


In [222]:
for param_tensor in model_hooks.actor_net.state_dict():
    print(param_tensor)

NameError: name 'model_hooks' is not defined